In this we will be tranning a CNN to classify dog or cat,
But before doing this we need to do Image-Preprocessing,
Here we have images of cats and dogs seperated in folders correctly, with correct name with serialno so
One solution is to extract label from the images and use it for supervised learning but this is not efficient approach
Keras Library has a solution for it we will see it in a while.

In Keras we just need to create simple structure which will help us in supervised learning from images without even 
accessing their names from the image labels.

First of all what we need to do is that we need to make training set and test set with images of the classes in the folder separetly, for ex like in training set folder we have to folder cats and dogs with images in them separated, so we have to do same with the test set, through this structure Keras will understand what is the correct label, you have done this before in the inception model in the transfer learning model

It is really important to use Feature Scaling in Deep Learning and CV applications we will use it later.

In [4]:
# Here we have do not need to do Data-Preprocessing so we will start from building our CNN model
# Part-1 : Building CNN Model

# Importing the important packages and libraries
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
# Here we use Sequential model to initialize our NN we have two ways one is Sequential and other is Graph
# In sequential we can add layers to the NN because CNN is also made from layers only
# As you can see that in the layer we have Convolution2D because we ae going to work on images and 
# if we want to apply CNN to videos then we can apply Convolution3D because we have time also being one dimension
# other is just the same that are required for building the CNN that is Convolution Layer, Pooling layer and then Flatten
# Dense package is used for fully Connected Layer


In [5]:
# This is just to verify that we have tensorflow Backend enabled because it is faster than theano
# We have /home/sparsh/anaconda2/envs/py35/etc/conda/activate.d 
# In this file we have changed the backend from theano to tensorflow
import keras
keras.backend.backend()

'tensorflow'

In [6]:
# Initializing the CNN
model = Sequential()

# Adding the Convolution Layer
model.add(Conv2D(32, (3, 3), input_shape=(64,64,3), activation='relu'))
# Here as we have studied in the CNN model we have used 32 feature detectors in the model and 3X3 is the size of the 
# feature detector and input_shape is the shape of the image we enter in the model for processing if you are using theano
# then the order would change see the documentation and here we are using activations as relu
# You should see the changes by googling them because the library are getting updated day by day

In [7]:
# Now Pooling the Feature Maps to reduce the size to reduce complexity without compromising performance
model.add(MaxPooling2D(pool_size=(2, 2)))
# 2X2 is the pool detector in general we take this because loosing information
# This step will reduce the size of the feature map and it will computionlly easy to process without lossing info

Why we are taking so much pain in creating a convolution layer and pooling layer, why dont we just flatten the input image just input it to the ANN?

The answer to above question is that if we input the image without convolution and pooling then we are just simply entering the all the pixels of the image and their is spatial realtion between the feature so classification becomes difficult and computationally expensive

But if we are doing the convolution layer and pooling we are extracating certain feature of the image and then we also reducing the size of the image and then we are applying the ANN but here we have also preserved the spatial oreintation of the image and we have also preserved some relationship between the pixel using the feature detectors.

SO HIGHLY RECOMMENDED TO USE CNN FOR IMAGE CLASSIFICATION

In [8]:
# Now we will increase the accuracy more by inserting a second convolution layer
model.add(Conv2D(32, (3, 3), activation='relu'))
# And we have to also repeat  the pooling step inorder to increase accuracy
model.add(MaxPooling2D(pool_size=(2, 2)))

In [9]:
# Flattening Step
model.add(Flatten())

In [10]:
# Now we have to add Fully Connected Hidden layer in the Network 
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
# Here output_dim is replaced with units ie number of the nodes in the layer
# Here we are using sigmoid function because we have binary classification but if more than 2 class we should use 
# softmax function

In [11]:
# Compile Step
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Now we will do Image Augmentation because it is important to prevent overfitting as we donot have a lot of images
# so we have to use this trick to train our model and prevent it from overfitting
# You can consider this as image processing step

# So what this image augmentation trick will do is that it will divide our training set into batches and then inside
# these batches the algorithm will rotate shift and trnasform the images and this will give us many more different 
# variations in the training set and we can train our model on different example this will make our model
# more GENRALIZED and it will predict better

# SUMMARY Image Augumentation
# It is a technique that allows us to enrich our dataset our training set without adding more images
# and therefore allows us to get better results with no or litte overfitting even with the small amounts of images



In [13]:
# This library is used for the technique discussed above
from keras.preprocessing.image import ImageDataGenerator

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
# Here we are creating a object of ImageDataGenerator class with these specifications to be applied in the 
# training set, Similary with test data
test_datagen = ImageDataGenerator(rescale=1./255)

# We know that our pixel value lie between 0-255 by using rescale we will pixel value between 0-1 because 1./255 will be
# multiplied to the pixel value, rescaling is very important, it is just changing pixel value not the size of the image


In [21]:
training_set = train_datagen.flow_from_directory('dataset/training_set', target_size=(64,64), batch_size=32, 
                                                 class_mode='binary')

test_set = train_datagen.flow_from_directory('dataset/test_set/', target_size=(64,64), batch_size=32, 
                                                 class_mode='binary')


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


(64, 64, 3)

In [13]:
# Here we are using fit generator method to fit our training set because our training data is generated by ImageData
# Generator
model.fit_generator(training_set, steps_per_epoch=8000, epochs=1, validation_data=test_set, validation_steps=2000)

Epoch 1/1
  42/8000 [..............................] - ETA: 4731s - loss: 0.7035 - acc: 0.5007

KeyboardInterrupt: 

In [21]:
# Now as we have trained our model we are going to predict for a single example
model.save('model.h5')

In [3]:
# We are using numpy and and image module from the preprocessing module of keras
# The image module is used to load the image which we are going to use for making the single prediction
# What we do is that we use image.img_to_array function to convert our test image into array of pixels
# Our test Image has dimensions like this 64 X 64 X 3
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)

# np.expand_dims will convert this image of 3 channels into one with axis onto axis=0 or row axis true
# And then we will feed this image into our model to predcit the test image we will use the predict method of our model
# to predict the category this can only be accesed when you have trained the model
# Now what this np.expand_dims will do is that it will add other dimension to the test_image
# that is now test_image is like 1 X 64 x 64 X 3
# this is because the model.predict method expect this kind of argument to predict the class
# so we need to add this dimensions
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

# Now the training_set.class_indices method is use to the know the indices of all the classes
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

[[[  54.   58.    7.]
  [  58.   63.    9.]
  [  64.   67.   10.]
  ..., 
  [ 136.  144.   71.]
  [ 140.  150.   77.]
  [ 139.  149.   78.]]

 [[  48.   54.    6.]
  [  51.   58.    7.]
  [  58.   63.    9.]
  ..., 
  [ 129.  137.   64.]
  [ 139.  149.   78.]
  [ 141.  151.   80.]]

 [[  48.   56.    7.]
  [  48.   56.    7.]
  [  54.   61.   10.]
  ..., 
  [ 123.  130.   63.]
  [ 136.  145.   80.]
  [ 140.  149.   82.]]

 ..., 
 [[  46.   55.   12.]
  [  42.   50.    9.]
  [  38.   49.    9.]
  ..., 
  [ 239.  205.  170.]
  [ 235.  209.  186.]
  [ 229.  202.  173.]]

 [[  50.   57.   15.]
  [  42.   50.    9.]
  [  44.   52.   11.]
  ..., 
  [ 234.  200.  162.]
  [ 236.  206.  178.]
  [ 234.  203.  174.]]

 [[  53.   59.   13.]
  [  43.   51.   10.]
  [  49.   56.   12.]
  ..., 
  [ 231.  195.  159.]
  [ 235.  213.  190.]
  [ 233.  206.  179.]]]
[[[[  54.   58.    7.]
   [  58.   63.    9.]
   [  64.   67.   10.]
   ..., 
   [ 136.  144.   71.]
   [ 140.  150.   77.]
   [ 139.  149.  

In [31]:
print(prediction)

dog
